In [0]:
import numpy as np
import _pickle as cPickle
import matplotlib.pyplot as plt
import torch
import helper
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch import nn
from torch import optim
from torchsummary import summary
from torchvision import datasets, transforms

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My Drive/modulation-dataset

/content/gdrive/My Drive/modulation-dataset


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# load dataset
data_dict = cPickle.load(open("RML2016.10b.dat",'rb'), encoding='bytes')
snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], data_dict.keys())))), [1,0])
data = []  
labels = []
for mod in mods:
    for snr in snrs:
        data.append(data_dict[(mod, snr)])
        for i in range(data_dict[(mod, snr)].shape[0]):  labels.append((mod, snr))
data = np.vstack(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)

(1200000, 2, 128)
(1200000, 2)


In [0]:
print(data.shape)

(1200000, 2, 128)


# ONLY RUN ONE CELL FROM THE FOLLOWING COMBINATIONS FOR MEMORY CONSIDERATIONS

#Derivative

In [0]:
data_first_derivative127 = np.zeros((2,127))
data_first_derivative = np.zeros((1200000,2,128))
zero = np.zeros((2,1 ))


##First derivative in time
for index in range(len(data)):
  data_first_derivative127 = np.diff(data[index], axis=1)
  data_first_derivative[index] = np.hstack((data_first_derivative127, zero))

print(data_first_derivative.shape)

(1200000, 2, 128)


# Integral

In [0]:
# integrate data

data_integ = np.zeros((1200000, 2, 128))
for i in range(len(data)):
  data_integ[i] = np.cumsum(data[i], axis = 1)

print(data_integ.shape)

(1200000, 2, 128)


# Raw and Derivative

In [0]:
data_first_derivative127 = np.zeros((2,127))
data_first_derivative = np.zeros((1200000,2,128))
zero = np.zeros((2,1 ))


##First derivative in time
for index in range(len(data)):
  data_first_derivative127 = np.diff(data[index], axis=1)
  data_first_derivative[index] = np.hstack((data_first_derivative127, zero))

print(data_first_derivative.shape)



raw_derivative = np.zeros((1200000, 4, 128))

for index in range(len(data)):
  raw_derivative[index] = np.vstack((data[index], data_first_derivative[index]))
  
print(raw_derivative.shape)

(1200000, 4, 128)


#Raw and Integral

In [0]:
# integrate data

data_integ = np.zeros((1200000, 2, 128))
for i in range(len(data)):
  data_integ[i] = np.cumsum(data[i], axis = 1)

print(data_integ.shape)



raw_integral = np.zeros((1200000, 4, 128))

for index in range(len(data)):
  raw_integral[index] = np.vstack((data[index], data_integ[index]))
  
print(raw_integral.shape)

# Integral and Derivative

In [0]:
# integrate data

data_integ = np.zeros((1200000, 2, 128))
for i in range(len(data)):
  data_integ[i] = np.cumsum(data[i], axis = 1)

print(data_integ.shape)



# differentiate data
data_first_derivative127 = np.zeros((2,127))
data_first_derivative = np.zeros((1200000,2,128))
zero = np.zeros((2,1 ))
##First derivative in time
for index in range(len(data)):
  data_first_derivative127 = np.diff(data[index], axis=1)
  data_first_derivative[index] = np.hstack((data_first_derivative127, zero))

print(data_first_derivative.shape)




# different combinations
data_diff_integ = np.zeros((1200000, 4, 128))

for index in range(len(data)):
  data_diff_integ[index] = np.vstack((data[index], data_integ[index]))

# All

In [0]:
# integrate data

data_integ = np.zeros((1200000, 2, 128))
for i in range(len(data)):
  data_integ[i] = np.cumsum(data[i], axis = 1)

print(data_integ.shape)



# differentiate data
data_first_derivative127 = np.zeros((2,127))
data_first_derivative = np.zeros((1200000,2,128))
zero = np.zeros((2,1 ))
##First derivative in time
for index in range(len(data)):
  data_first_derivative127 = np.diff(data[index], axis=1)
  data_first_derivative[index] = np.hstack((data_first_derivative127, zero))

print(data_first_derivative.shape)





data_all = np.zeros((1200000, 4, 128))

for index in range(len(data)):
  intermediate = np.vstack((data[index], data_integ[index]))
  data_all[index] = np.vstack((intermediate, data_first_derivative[index]))

# Splitting dataset into training and test set (50%-50%)

In [0]:
x_train_validate, x_test, y_train_validate, y_test = train_test_split(data, labels, test_size=0.5, shuffle = True)

print(x_train_validate.shape)
print(x_test.shape)
print(y_train_validate.shape)
print(y_test.shape)

#Validation 5% from training set

x_train, x_validate, y_train, y_validate = train_test_split(x_train_validate, y_train_validate, test_size= 0.05, shuffle=False)

print(x_train.shape)
print(x_validate.shape)
print(y_train.shape)
print(y_validate.shape)

(600000, 2, 128)
(600000, 2, 128)
(600000, 2)
(600000, 2)
(570000, 2, 128)
(30000, 2, 128)
(570000, 2)
(30000, 2)


## Mapping Categorical labels

In [0]:
one_hot_encoder = preprocessing.OneHotEncoder()

one_hot_encoder.fit(y_train[:, 0].reshape(-1,1))
print(y_train[0])
y_train = one_hot_encoder.transform(y_train[:, 0].reshape(-1,1)).todense()
print(one_hot_encoder.inverse_transform(y_train[0]))

one_hot_encoder.fit(y_validate[:, 0].reshape(-1,1))
print(y_validate[0])
y_validate = one_hot_encoder.transform(y_validate[:, 0].reshape(-1,1)).todense()
print(one_hot_encoder.inverse_transform(y_validate[0]))

one_hot_encoder.fit(y_test[:, 0].reshape(-1,1))
print(y_test[0])
y_test = one_hot_encoder.transform(y_test[:, 0].reshape(-1,1)).todense()
print(one_hot_encoder.inverse_transform(y_test[0]))


[b'GFSK' b'-2']
[[b'GFSK']]
[b'BPSK' b'-4']
[[b'BPSK']]
[b'QAM16' b'-12']
[[b'QAM16']]


## Building NN

In [0]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.hidden1 = nn.Linear(256, 400)
        self.hidden2 = nn.Linear(400, 512)
        #self.hidden3 = nn.Linear(768, 512)
        #self.hidden4 = nn.Linear(512, 10)
        # Output layer, 10 units - one for each mod type
        self.output = nn.Linear(512, 10)
        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)
        
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        #print(x.shape)
        # Pass the input tensor through each of our operations
        x = self.dropout(F.relu(self.hidden1(x)))
        x = self.dropout(F.relu(self.hidden2(x)))
        #x = self.dropout(F.relu(self.hidden3(x)))
        #x = self.dropout(F.relu(self.hidden4(x)))
        x = self.output(x)
        
        
        return x

# Converting  from numpy to tensor


In [0]:
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

x_validate = torch.from_numpy(x_validate)
y_validate  = torch.from_numpy(y_validate)

x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)

# Change dataset to loader

In [0]:
batch_size = 2048
num_workers = 0


x_train_loader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=False)
y_train_loader = torch.utils.data.DataLoader(y_train, batch_size=batch_size, shuffle=False)

x_validate_loader = torch.utils.data.DataLoader(x_validate, batch_size=batch_size, shuffle=False)
y_validate_loader = torch.utils.data.DataLoader(y_validate, batch_size=batch_size, shuffle=False)

x_test_loader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, shuffle=False)
y_test_loader = torch.utils.data.DataLoader(y_test, batch_size=batch_size, shuffle=False)

# Choose CPU or GPU

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Creating a model

In [0]:
model = Network()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
criterion = nn.CrossEntropyLoss()
print(model)

model.to(device)

# Training the model

In [0]:
n_epochs = 100
steps = 0
​
batch_size = 2048
num_workers = 0
​
y_train = y_train.long()
y_validate = y_validate.long()
y_test = y_test.long()
​
# divide data into batches 
x_train_loader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, num_workers=num_workers)
y_train_loader = torch.utils.data.DataLoader(y_train, batch_size=batch_size, num_workers=num_workers)
​
x_validate_loader = torch.utils.data.DataLoader(x_validate, batch_size=batch_size, num_workers=num_workers)
y_validate_loader = torch.utils.data.DataLoader(y_validate, batch_size=batch_size, num_workers=num_workers)
​
x_test_loader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, num_workers=num_workers)
y_test_loader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, num_workers=num_workers)
​
train_losses, validate_losses = [], []
for e in range(n_epochs):
    running_loss = 0
    for samples, labels in zip(x_train_loader, y_train_loader):
      
        # Move input and label tensors to the default device
        samples, labels = samples.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        probabilities = model(samples)
        # print(probabilities.dim())
        # print(label.dim())
        
        loss = criterion(probabilities, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        validate_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for samples, labels in zip(x_validate_loader, y_validate_loader):
              
                # Move input and label tensors to the default device
                samples, labels = samples.to(device), labels.to(device)
                
                
                probabilities = model(samples)
                validate_loss += criterion(probabilities, torch.max(labels, 1)[1])
                
                top_p, top_class = probabilities.topk(1, dim=1)
                equals = top_class == torch.max(labels, 1)[1].view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(y_train_loader))
        validate_losses.append(validate_loss/len(y_test_loader))
​
        print("Epoch: {}/{}.. ".format(e+1, n_epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Validation Loss: {:.3f}.. ".format(validate_losses[-1]),
              "Validation Accuracy: {:.3f}".format(accuracy/len(y_validate_loader)))

In [0]:
plt.plot(train_losses, label='Training loss')

In [0]:
plt.plot(validate_losses, label='Validation loss')

# Build CNN 

In [0]:
class CNN(nn.Module):
  def __init__(self):
      super().__init__()
      # convolutional layer (sees 2x128x1 tensor)
      self.conv1 = nn.Conv2d(1, 64, (1,3), padding=0)
      # convolutional layer 
      self.conv2 = nn.Conv2d(64, 16, (2,3), padding=0)

      self.fc1 = nn.Linear(3*128*16 , 128)
      # linear layer (128 -> 10)
      self.fc2 = nn.Linear(128, 10)
      # dropout layer (p=0.25)
      self.dropout = nn.Dropout(0.25)
    
  def forward(self, x):
      pad1 = (1, 1)
      pad2 = (1, 1, 1, 1)
      x = F.pad(x, pad1, "constant", 0)
      x = F.relu(self.conv1(x))
      #print(x.shape)
      x = F.pad(x, pad2, "constant", 0)
      x = F.relu(self.conv2(x))
      #print(x.shape)
      x = x.view(x.shape[0], -1)
      x = self.dropout(F.relu(self.fc1(x)))
      x = self.fc2(x)
     
      return x

# Creating CNN model

In [0]:
model_CNN = CNN()
optimizer = optim.Adam(model_CNN.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

criterion = nn.CrossEntropyLoss()
#move the model to device
model_CNN.to(device)

# Train the CNN model

In [0]:
n_epochs = 50
steps = 0
valid_loss_min = np.Inf # track change in validation loss

train_losses, validate_losses = [], []
for e in range(n_epochs):
    running_loss = 0
    for samples, labels in zip(x_train_loader, y_train_loader):
      
        # Move input and label tensors to the default device
        samples, labels = samples.to(device), labels.to(device)
        optimizer.zero_grad()
        probabilities = model_CNN(torch.unsqueeze(samples, 1))  # add a depth size 1
        
        loss = criterion(probabilities, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        validate_loss = 0
        accuracy = 0
        correct = 0
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model_CNN.eval()
            for samples, labels in zip(x_validate_loader, y_validate_loader):
              
                # Move input and label tensors to the default device
                samples, labels = samples.to(device), labels.to(device)
                
                
                probabilities = model_CNN(torch.unsqueeze(samples, 1))  # add a depth size 1
                validate_loss += criterion(probabilities, torch.max(labels, 1)[1])    
                
                top_p, top_class = probabilities.topk(1, dim=1)
                equals = top_class == torch.max(labels, 1)[1].view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                correct += torch.sum(equals.type(torch.FloatTensor))
        
        model_CNN.train()
        
        train_losses.append(running_loss/len(y_train_loader))
        validate_losses.append(validate_loss/len(y_validate_loader))
        scheduler.step(validate_losses[-1])
        
        if validate_losses[-1] < valid_loss_min:
          torch.save(model_CNN.state_dict(), 'model_CNN_RAW.pt')
          print("saving model...")
          valid_loss_min = validate_losses[-1]

        print("Epoch: {}/{}.. ".format(e+1, n_epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Validation Loss: {:.3f}.. ".format(validate_losses[-1]),
              "Validation Accuracy: {:.3f}".format(accuracy/len(y_validate_loader)))
        #print("correct = ".correct)

In [0]:

batch_size = 2048
num_workers = 0


x_train = x_train.float()
y_train = y_train.float()
x_validate = x_validate.float()
y_validate = y_validate.float()
x_test = x_test.float()
y_test = y_test.float()

x_train_loader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=False)
y_train_loader = torch.utils.data.DataLoader(y_train, batch_size=batch_size, shuffle=False)

x_validate_loader = torch.utils.data.DataLoader(x_validate, batch_size=batch_size, shuffle=False)
y_validate_loader = torch.utils.data.DataLoader(y_validate, batch_size=batch_size, shuffle=False)

x_test_loader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, shuffle=False)
y_test_loader = torch.utils.data.DataLoader(y_test, batch_size=batch_size, shuffle=False)